In [1]:
import pandas as pd
import numpy as np
import regex as re
from io import StringIO


In [2]:
pd_csv = pd.read_csv("output-9-pandas.csv")


In [3]:
pd_csv = pd_csv.iloc[:, 1:]
# Resetting index
pd_csv.reset_index()
# Find out where portfolio company is
for col in pd_csv.columns:
    pd_csv[col] = pd_csv[col].astype(str).str.replace('  ', ' ')
    portfolio_index_list = pd_csv.index[pd_csv[col].astype(str).str.contains("Portfolio Company")].to_list()
    if (len(portfolio_index_list) > 0):
        break
portfolio_index = portfolio_index_list[0]
# Checking if all the column are in the row with portfolio and adding them if they are not.
if (pd_csv.iloc[portfolio_index].str.contains('Principal').any() == False):
    for j in range(pd_csv.shape[1]):
        if (pd_csv.iat[portfolio_index - 1, j] == 'Principal'):
            pd_csv.iat[portfolio_index, j] = 'Principal'
        elif (pd_csv.iat[portfolio_index - 1, j] == 'Cost'):
            pd_csv.iat[portfolio_index, j] = 'Cost'
        elif (pd_csv.iat[portfolio_index - 1, j] == 'Fair'):
            pd_csv.iat[portfolio_index, j] = 'Fair Value'
# Removing rows before portfolio index
if (pd_csv.iloc[portfolio_index].str.contains('Principal').any() == True):
    pd_csv = pd_csv.iloc[portfolio_index:]

# Identifying levels in the dataframe
listOfLevels = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (pd.notnull(pd_csv.iat[i, j])):
            # print(pd_csv.iat[i, j])
            if (('Investments' in str(pd_csv.iat[i, j])) and
                    ('Subtotal' not in pd_csv.iat[i, j]) and
                    ('Total' not in pd_csv.iat[i, j])):
                if (pd_csv.iloc[i].isnull().sum() == pd_csv.shape[1] - 1):
                    # print(pd_csv.iloc[i])
                    listOfLevels.append(i)

# Adding the Levels to an array
newValues = []
newValues.append("Levels")
for i in range(len(listOfLevels)):
    beginIndex = listOfLevels[i]
    if (i == len(listOfLevels) - 1):
        endIndex = len(pd_csv)
    else:
        endIndex = listOfLevels[i + 1]
    value = pd_csv.iat[beginIndex, 0]
    for x in range(beginIndex, endIndex):
        newValues.append(value)
# Adding the array to the df
pd_csv.insert(0, 'Levels', newValues)

# Finding Subtotals
pd_csv.reset_index()
subtotal_list = list()
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if (('Subtotal' in str(pd_csv.iat[i, j]))):
            # print(pd_csv.iloc[i,:])
            subtotal_list.append(i)

print("Subtotal", subtotal_list)
# Dropping the subtotal rows
pd_csv.drop(pd_csv.index[subtotal_list], inplace=True)

# Finding Totals
total_list = list()
for i in range(pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        if ('Total' in str(pd_csv.iat[i, j])):
            total_list.append(i)
# Dropping everything after the total rows
try:
    pd_csv = pd_csv.iloc[:total_list[0]]
except:
    print("Total not found")

# Adding company for the middle values where it might be missed out
for i in range(2, pd_csv.shape[0]):
    for j in range(pd_csv.shape[1]):
        row_alphabet_check = False
        if (pd.isnull(pd_csv.iat[i, j]) or (pd_csv.iat[i, j] == 'nan')):
            # print("Index",i,j)
            if (i > 0):
                # print(pd_csv.iat[i-1,j])
                if (pd.isnull(pd_csv.iat[i - 1, j])):
                    1
                else:
                    if (re.search('[a-zA-Z]', str(pd_csv.iat[i - 1, j]))):
                        for k in range(pd_csv.shape[1]):
                            if (re.search('[a-zA-Z]', str(pd_csv.iat[i, k])) and pd.notnull(pd_csv.iat[i, k])):
                                # print("Alpha Check",pd_csv.iat[i,k])
                                row_alphabet_check = True
                                break
                        if (row_alphabet_check):
                            # print(pd_csv.iat[i-1,j])
                            pd_csv.iat[i, j] = pd_csv.iat[i - 1, j]

# Identifying index of Principal
i = 0
for j in range(pd_csv.shape[1]):
    if ('Principal' in str(pd_csv.iat[i, j])):
        principal_index = j
# Extracting the column data after principal
pd_csv_principal = pd_csv.iloc[:, principal_index - 1:]
# Sweeping everyhing to left in extracted df
pd_csv_principal = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_principal.to_csv())))
pd_csv_principal = pd_csv_principal.iloc[:, 1:]
# Dropping columns with Nulls
perc = 95  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv_principal.shape[0] + 1)
pd_csv_principal = pd_csv_principal.dropna(axis=1,
                                           thresh=min_count)
pd_csv_principal.update(
    pd_csv_principal.loc[pd_csv_principal.iloc[:, -1].isna(), :].shift(axis=1).replace(np.nan, 'na')
)
pd_csv_principal.replace('na', np.nan, inplace=True)
pd_csv = pd_csv.iloc[:, :principal_index - 1].reset_index()
# Concatting everything together
pd_csv = pd.concat([pd_csv, pd_csv_principal], axis=1)

# Sweeping to left for member units
pd_csv_sweep_left = pd_csv.iloc[:, :(principal_index + 2)]
pd_csv_sweep_left = pd.read_csv(StringIO(re.sub(',+', ',', pd_csv_sweep_left.to_csv())))
pd_csv_sweep_right = pd_csv.iloc[:, principal_index + 2:]
pd_csv = pd.concat([pd_csv_sweep_left, pd_csv_sweep_right], axis=1)

perc = 100  # Like N %
min_count = int(((100 - perc) / 100) * pd_csv.shape[0] + 1)
# Dropping columns which have 100% NaN
pd_csv = pd_csv.dropna(axis=1,
                       thresh=min_count)

new_header = pd_csv.iloc[0]  # grab the first row for the header
pd_csv = pd_csv[1:]  # take the data less the header row
pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header

pd_csv = pd_csv.iloc[:, 1:]
pd_csv = pd_csv.replace("~", ",", regex=True)
pd_csv = pd_csv.iloc[:, 1:]

# Dropping columns which NaN as column names
pd_csv = pd_csv.loc[:, pd_csv.columns.notna()]
pd_csv = pd_csv.dropna(thresh=len(pd_csv.columns) - 1)
pd_csv = pd_csv.fillna("")
try:
    for col in pd_csv.columns:
        # print("Column", col)
        pd_csv[col] = pd_csv[col].astype(str).str.replace(r"\(\d+\)", "", regex=True)
except:
    print("Bracket digits not removed")

Subtotal [5, 17, 147]


C:\Users\jaiso\AppData\Local\Temp\ipykernel_14732\58683545.py:136: FutureWarning: The default value of regex will change from True to False in a future version.
  pd_csv.columns = new_header.str.replace(r"\(\d+\)", "")  # set the header row as the df header


In [7]:
pd_csv_principal

,2,3,4,5
0,Type of Investment (2) (3),Principal (7),Cost (7),Fair Value
1,NaN,NaN,NaN,NaN
2,"LIBOR Plus 9.00% (Floor 1.00%), Current Coupon...",8147,7996.0,7996.0
3,NaN,"Member Units (2,896 shares)",6435,6435.0
4,NaN,"Member Units (2,896 shares)",14431,14431.0
...,...,...,...,...
140,NaN,"Member Units (2,090,001 shares)",6000,6000.0
141,NaN,"Warrants (407,408 equivalent shares)",600,600.0
142,NaN,"Warrants (407,408 equivalent shares)",13410,13410.0
143,"LIBOR Plus 8.00% (Floor 1.00%), Current Coupon...",6468,6409,6500.0
